In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

PRE-EXISTING API TOOLS

In [57]:
wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=wrapper)

In [7]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper


PRE-EXISTING API TOOLS 

In [10]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

CREATING NEW API TOOL

In [13]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [19]:
loader = WebBaseLoader("https://python.langchain.com/docs/introduction/")
embedding = OllamaEmbeddings(model= "llama2")
documents = loader.load()
text_splitter  = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
texts = text_splitter.split_documents(documents[:50])


In [20]:

vectorstore = FAISS.from_documents(texts, embedding)

In [21]:
retriever = vectorstore.as_retriever()


In [67]:
from langchain.tools.retriever import create_retriever_tool
langchain_tool = create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")


In [77]:
from langchain.agents import Tool
from langchain.utilities import WikipediaAPIWrapper

wikipedia = WikipediaAPIWrapper()
arxivapi = ArxivAPIWrapper()
# Create dummy tools
wiki = WikipediaQueryRun(api_wrapper=wikipedia)
arxiv = ArxivQueryRun(api_wrapper=arxivapi)


retriever_tool = Tool(
    name="DocumentRetriever",
    func=lambda q: retriever.get_relevant_documents(q),
    description="Use this to fetch documents related to a user query."
)

tools = [retriever_tool, wiki, arxiv]

from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=3,
)

response = agent.run("What is LangChain?")
print(response)



> Entering new AgentExecutor chain...
Question: What is LangChain?
Thought: Let me check if there are any relevant documents on Arxiv or Wikipedia that can help me understand what LangChain is.
Action: arxiv
Action Input: "LangChain"
Observation: Published: 2025-01-27
Title: From Prompt Injections to SQL Injection Attacks: How Protected is Your LLM-Integrated Web Application?
Authors: Rodrigo Pedro, Daniel Castro, Paulo Carreira, Nuno Santos
Summary: Large Language Models (LLMs) have found widespread applications in various
domains, including web applications, where they facilitate human interaction
via chatbots with natural language interfaces. Internally, aided by an
LLM-integration middleware such as Langchain, user prompts are translated into
SQL queries used by the LLM to provide meaningful responses to users. However,
unsanitized user prompts can lead to SQL injection attacks, potentially
compromising the security of the database. Despite the growing interest in
prompt injectio

In [80]:
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain import hub

retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="document_retriever",
    description="Retrieves documents based on user queries."
)

tools = [retriever_tool, wiki, arxiv]

prompt = hub.pull("hwchase17/openai-functions-agent")

agent = create_openai_functions_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
response = agent_executor.invoke({"input": "What is LangChain?"})
print(response)




> Entering new AgentExecutor chain...


ValueError: This output parser only works on ChatGeneration output